In [3]:
import numpy as np
import pandas as pd

In [4]:
def areas(box):
    return (box[2] - box[0]) * (box[3] - box[1])

def intersection_over_union(boxes):
    assert(len(boxes) == 8)
    boxA = boxes[:4].values
    boxB = boxes[4:].values
    
    boxAArea = areas(boxA)
    boxBArea = areas(boxB)
    
    if (boxAArea == 0 or boxBArea == 0):
        return 0
        
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    interArea = max(0, xB - xA) * max(0, yB - yA)

    
    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou
box_columns = ['Xmin','Ymin','Xmax','Ymax']

In [24]:
t_d = pd.read_csv('train_data.csv')
t_d['pxs'] = (t_d['Xmax'] - t_d['Xmin'])*(t_d['Ymax'] - t_d['Ymin'])
pxs = t_d.groupby("itemId")['pxs'].mean()
t_d = t_d.merge(pxs, on=["itemId"])
t_d['pxs_ratio'] = t_d['pxs_x'] / t_d['pxs_y']
train_answers = pd.read_csv('train_answers.csv')
ids = list(set(t_d['itemId']))
users_rating = pd.read_csv("users_rating.csv")
users_rating = users_rating.set_index('userId')

In [42]:
train_full = t_d[t_d['pxs_ratio'] > .5].merge(train_answers, on=["itemId"])
train_full['iou'] = train_full[['Xmin','Ymin', 'Xmax', 'Ymax', 'Xmin_true',\
      'Ymin_true', 'Xmax_true','Ymax_true']].apply(intersection_over_union, axis=1)
train_full.groupby("userId")['iou'].mean().to_csv('users_rating.csv', header=True)

In [43]:
users_rating = pd.read_csv("users_rating.csv")
users_rating = users_rating.set_index('userId')
quorum = pd.DataFrame(np.zeros((len(ids), 5), dtype=int), index=ids, columns=box_columns + ['itemId'])
for i in ids:    
    uids = t_d[t_d['itemId'] == i]['userId']
    uid = users_rating.loc[uids].idxmax()['iou']
    ti = t_d.loc[t_d['itemId'] == i].loc[t_d['userId'] == uid][box_columns].max()
    for y in box_columns:
        quorum.loc[i][y] = ti[y]   
    quorum.loc[i]['itemId'] = i
data = quorum.merge(train_answers, on=['itemId'])
data["iou"] = data[['Xmin','Ymin', 'Xmax', 'Ymax', 'Xmin_true',\
      'Ymin_true', 'Xmax_true','Ymax_true']].apply(intersection_over_union, axis=1)
print(data["iou"].mean())    

/home/vasste/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  


0.5515876066326573
